In [1]:
from myutils.nerdssData import Reaction_Name, _Reaction_nerdss, Write_Inp, Write_CrdInp
from myutils import utils
import numpy as np
from pathlib import Path

In [2]:
def notInNucleosomes(crd, nucCrds, nucSizes):
    for i in range(len(nucCrds)):
        if abs(crd - nucCrds[i]) <= nucSizes[i]:
            return False
    return True

def generate_N_crd(N_nonspec, xlim):
    NCrds = np.random.rand(N_nonspec)*(xlim[1]-xlim[0])+xlim[0]
    return NCrds

def generate_P_crd(Np, xlim, y, z, sigma):
    aveDistance = (xlim[1]-xlim[0])/(Np+1)
    pCrds = []
    for i in range(Np):
        if i%2 == 0:
            pCrds.append([ xlim[0] + (i+1)*aveDistance, y, z])
        else:
            pCrds.append([ xlim[0] + (i-1+1)*aveDistance+sigma, y, z])
                
    return pCrds

# for comparing two segments

## dimers

In [3]:
def onRate_macro2micro(ka_macro, D, sigma):
        if ka_macro < 1e-10:
            return 0
        else:
            return (1/ka_macro - 1/(4*np.pi*D*sigma))**-1

def get_static_Parms(
    kbPFixed=0.4, DP3D=1.5, DN1D=1.0, PPsigma=1.4, 
    lengthScale=50, dt=1, area3Dto1D=10, 
    kaPS_macro = 1000, kaPN_macro = 200
):
    
    C0 = 0.6022

    DP1D = (1/DP3D + 1/DN1D)**(-1)
    konMax = (4*np.pi*(2*DP3D)*PPsigma)
    print('DP1D: %.3f\tKonD3d: %.3f\tKPPmax (M^-1): %.1e'%(DP1D, konMax, konMax/(kbPFixed*1e-6)*C0))
    kaPS = onRate_macro2micro(kaPS_macro*1e-6, DP3D, 1)
    kaPN = onRate_macro2micro(kaPN_macro*1e-6, DP3D, 1)

    Rmax = PPsigma + 3 * np.sqrt( 6*DP3D*2 * dt )
    
    passed = True
    if Rmax > WaterBox[0]/2:
        print(f'Diffusion per step too large to stay in waterbox.x: Rmax={Rmax:.1f}, x/2={WaterBox[0]/2:.1f}')
        passed = True
    if Rmax > WaterBox[1]/2:
        print(f'Diffusion per step too large to stay in waterbox.y: Rmax={Rmax:.1f}, x/2={WaterBox[1]/2:.1f}')
        passed = True
    if Rmax > WaterBox[2]/2:
        print(f'Diffusion per step too large to stay in waterbox.z: Rmax={Rmax:.1f}, x/2={WaterBox[2]/2:.1f}')
        passed = True
    if passed:
        print('Diffusion and time step fit in waterbox.')
        
    L = lengthScale*21/2 - PPsigma # nm
    Diff = DP1D* 2 # nm^2/us
    onRateDiff = (L/3/Diff)**(-1)
    print(f'(L/(3D))^(-1) = {onRateDiff:.1e} nm/us')
    
    return kaPS, kaPN

In [4]:
lengthScale = 2
WaterBox = [21*lengthScale*2 + 22, 31.623, 31.623]
excLength = 5.51
kbPFixed = 0.4
ppSigma = 1.4
DP3D = 1.5

# manualy set the positions of targets
quarter = WaterBox[0]/4
SCrds_CB = [-quarter-1, -quarter+1, int(2*quarter*2/3)/2, int(2*quarter*4/3)/2]
# automatically calculate the N crds and nuc crds
N_non = int(12*lengthScale - len(SCrds_CB)/2)
NCrds = np.concatenate([
    generate_N_crd(N_non, [-WaterBox[0]/2+excLength, -excLength]), 
    generate_N_crd(N_non, [excLength, WaterBox[0]/2-excLength])
])
nucCrds = [-WaterBox[0]/2, WaterBox[0]/2, 0] 
print('Nuc:', nucCrds)
print('S:', SCrds_CB)
# calculate fixed parameters
kaPS, kaPN = get_static_Parms(
    kbPFixed, PPsigma=ppSigma, DP3D=DP3D, kaPN_macro=20000, kaPS_macro=100000
)
print('kaPS', kaPS, 'kaPN', kaPN)

Nuc: [-53.0, 53.0, 0]
S: [-27.5, -25.5, 17.5, 35.0]
DP1D: 0.600	KonD3d: 52.779	KPPmax (M^-1): 7.9e+07
Diffusion and time step fit in waterbox.
(L/(3D))^(-1) = 6.9e-03 nm/us
kaPS 0.10053334596519763 kaPN 0.020021243198812908


In [5]:
C0 = 0.6022
def rlog(x, acc=2):
    if x==0:
        return '-inf'
    else:
        if acc==0:
            return int(np.log10(x))
        else:
            return round(np.log10(x), acc)

equi_parms = [
    [10/C0, 0/C0],
    [10/C0, 1e1/C0], #
    [10/C0, 1e2/C0], #
    [10/C0, 1e3/C0], #
    [10/C0, 1e4/C0], #
    [10/C0, 1e5/C0], #
    [10/C0, 1e6/C0], #
] # M^-1 -> nm^3

N_repeat = 48
accBind = 1 # accelarate binding to relax from bound state
pfolder = './twoSegments_4pro/'

for KPN, KPP in equi_parms:
    KPS = KPN*1e3
    kbPS = kaPS*1e6 / KPS
    kbPN = kaPN*1e6 / KPN
    if KPP == 0:
        kbPP = 0
        kaPP = 0
    else:
        kbPP = kbPFixed
        kaPP = onRate_macro2micro(KPP*kbPP*1e-6, 2*DP3D, ppSigma)
        kbPP = kaPP / KPP * 1e6
        
    print('kaP: %.2e'%kaPP, 'kbP: %.2e'%kbPP, 'kbN: %.2e'%kbPN, 'kbS: %.2e'%kbPS)
    nItr = 1001
    SCrds = SCrds_CB
    
    for rn in range(N_repeat):
        if KPP == 0:
            namedir = f'{pfolder}/kpp0kpnE{rlog(KPN*C0,1)}/{rn}/'
        else:
            namedir = f'{pfolder}/kppE{rlog(KPP*C0,1)}kpnE{rlog(KPN*C0,1)}/{rn}/'
        parmFile = f'''# two proteins discover between two segments

start parameters
    nItr = {int(nItr)} #iterations
    timeStep = 1 # us, the maximum timestep calculated from kbPN
    timeWrite = 1e5
    pdbWrite = 1e5
    trajWrite = {int((nItr/1e1))}
    assocdissocwrite = true
    overlapSepLimit = 1.0
end parameters

start boundaries
    WaterBox = {WaterBox} # VtoL=4000
end boundaries

start molecules
    P : 4
    N : {len(NCrds)}
    S : {len(SCrds)}
    nuc : {len(nucCrds)}
end molecules

start reactions

    N(ref) + nuc(ref) -> N(ref!1).nuc(ref!1)
    onRate3Dka = 0
    offRatekb = 0
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 5.5
    assocAngles = [nan, nan, nan, nan, nan]
    loopcoopfactor = 1
    bindRadSameCom = 1.1

    P(ref) + nuc(ref) -> P(ref!1).nuc(ref!1)
    onRate3Dka = 0
    offRatekb = 0
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 5.5
    assocAngles = [nan, nan, nan, nan, nan]
    loopcoopfactor = 1
    bindRadSameCom = 1.1    

    S(re1) + S(re2) -> S(re1!1).S(re2!1)
    onRate3Dka = 1e6
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.8001
    assocAngles = [M_PI, M_PI, nan, nan, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.00001

    S(bs) + P(dbs) <-> S(bs!1).P(dbs!1)
    onRate3Dka = {kaPS}
    offRatekb = {kbPS} # Keq = 10^{rlog(1e3*KPN*C0)}
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.00
    assocAngles = [0.785398, 2.356194, 0.000000, M_PI, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10

    N(bs) + P(dbn) <-> N(bs!1).P(dbn!1)
    onRate3Dka = {kaPN}
    offRatekb = {kbPN} # Keq = 10^{rlog(KPN*C0)}
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.000007
    assocAngles = [1.604246, 0.948003, 1.221389, -2.514197, -1.261396]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10
    
    P(pp1) + P(pp2) <-> P(pp1!1).P(pp2!1)
    onRate3Dka = {kaPP} 
    offRatekb = {kbPP} # Keq = 10^{rlog(KPP*C0)}
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.4
    assocAngles = [2.111216, 2.111216, -1.570796, 1.570796, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10

end reactions
'''
        # create directory and write parameters
        Path(namedir).mkdir(parents=True, exist_ok=True)
        with open(f'./{namedir}/parms.inp', 'w') as f:
            f.write(parmFile)
        # write coordinates
        crdFileName = f'./{namedir}/fixCoordinates.inp'
        fixCoords={'S':SCrds,  'N':NCrds, 'nuc':nucCrds}
        Write_CrdInp(crdFileName, fixCoords, WaterBox)

kaP: 0.00e+00 kbP: 0.00e+00 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.64e-06 kbP: 4.00e-01 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.64e-05 kbP: 4.00e-01 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.64e-04 kbP: 4.00e-01 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.64e-03 kbP: 4.00e-01 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.65e-02 kbP: 4.01e-01 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.73e-01 kbP: 4.05e-01 kbN: 1.21e+03 kbS: 6.05e+00


## multimer in a row

In [17]:
def onRate_macro2micro(ka_macro, D, sigma):
        if ka_macro < 1e-10:
            return 0
        else:
            return (1/ka_macro - 1/(4*np.pi*D*sigma))**-1

def get_static_Parms(
    kbPFixed=0.4, DP3D=1.5, DN1D=1.0, PPsigma=1.4, 
    lengthScale=50, dt=1, area3Dto1D=10, 
    kaPS_macro = 1000, kaPN_macro = 200
):
    
    C0 = 0.6022

    DP1D = (1/DP3D + 1/DN1D)**(-1)
    konMax = (4*np.pi*(2*DP3D)*PPsigma)
    print('DP1D: %.3f\tKonD3d: %.3f\tKPPmax (M^-1): %.1e'%(DP1D, konMax, konMax/(kbPFixed*1e-6)*C0))
    kaPS = onRate_macro2micro(kaPS_macro*1e-6, DP3D, 1)
    kaPN = onRate_macro2micro(kaPN_macro*1e-6, DP3D, 1)

    Rmax = PPsigma + 3 * np.sqrt( 6*DP3D*2 * dt )
    
    passed = True
    if Rmax > WaterBox[0]/2:
        print(f'Diffusion per step too large to stay in waterbox.x: Rmax={Rmax:.1f}, x/2={WaterBox[0]/2:.1f}')
        passed = True
    if Rmax > WaterBox[1]/2:
        print(f'Diffusion per step too large to stay in waterbox.y: Rmax={Rmax:.1f}, x/2={WaterBox[1]/2:.1f}')
        passed = True
    if Rmax > WaterBox[2]/2:
        print(f'Diffusion per step too large to stay in waterbox.z: Rmax={Rmax:.1f}, x/2={WaterBox[2]/2:.1f}')
        passed = True
    if passed:
        print('Diffusion and time step fit in waterbox.')
        
    L = lengthScale*21/2 - PPsigma # nm
    Diff = DP1D* 2 # nm^2/us
    onRateDiff = (L/3/Diff)**(-1)
    print(f'(L/(3D))^(-1) = {onRateDiff:.1e} nm/us')
    
    return kaPS, kaPN

In [18]:
lengthScale = 3
WaterBox = [21*lengthScale*2 + 22, 31.623, 31.623]
excLength = 5.51
kbPFixed = 0.4
ppSigma = 1.4
DP3D = 1.5

# manualy set the positions of targets
quarter = WaterBox[0]/4
SCrds_CB = [-quarter-2, -quarter, -quarter+2, quarter/2, quarter, quarter*3/2]
# automatically calculate the N crds and nuc crds
N_non = int(12*lengthScale - len(SCrds_CB)/2)
NCrds = np.concatenate([
    generate_N_crd(N_non, [-WaterBox[0]/2+excLength, -excLength]), 
    generate_N_crd(N_non, [excLength, WaterBox[0]/2-excLength])
])
nucCrds = [-WaterBox[0]/2, WaterBox[0]/2, 0] 
print('Nuc:', nucCrds)
print('S:', SCrds_CB)
# calculate fixed parameters
kaPS, kaPN = get_static_Parms(
    kbPFixed, PPsigma=ppSigma, DP3D=DP3D, kaPN_macro=20000, kaPS_macro=100000
)

Nuc: [-74.0, 74.0, 0]
S: [-39.0, -37.0, -35.0, 18.5, 37.0, 55.5]
DP1D: 0.600	KonD3d: 52.779	KPPmax (M^-1): 7.9e+07
Diffusion and time step fit in waterbox.
(L/(3D))^(-1) = 6.9e-03 nm/us


In [19]:
C0 = 0.6022
def rlog(x, acc=2):
    if x==0:
        return '-inf'
    else:
        if acc==0:
            return int(np.log10(x))
        else:
            return round(np.log10(x), acc)

equi_parms = [
    [10/C0, 0/C0], #
    [10/C0, 1e6/C0], #
] # M^-1 -> nm^3

N_repeat = 15
accBind = 1 # accelarate binding to relax from bound state
pfolder = './twoSegments_trimer/'

for KPN, KPP in equi_parms:
    KPS = KPN*1e3
    kbPS = kaPS*1e6 / KPS
    kbPN = kaPN*1e6 / KPN
    if KPP == 0:
        kbPP = 0
        kaPP = 0
    else:
        kbPP = kbPFixed
        kaPP = onRate_macro2micro(KPP*kbPP*1e-6, 2*DP3D, ppSigma)
        kbPP = kaPP / KPP * 1e6
        
    print('kaP: %.2e'%kaPP, 'kbP: %.2e'%kbPP, 'kbN: %.2e'%kbPN, 'kbS: %.2e'%kbPS)
    nItr = max(int(1e9), 10**int(np.log10(1/kbPS)+4))
    SCrds = SCrds_CB
    
    for rn in range(N_repeat):
        if KPP == 0:
            namedir = f'{pfolder}/kpp0kpnE{rlog(KPN*C0,1)}/{rn}/'
        else:
            namedir = f'{pfolder}/kppE{rlog(KPP*C0,1)}kpnE{rlog(KPN*C0,1)}/{rn}/'
        parmFile = f'''# two proteins discover between two segments

start parameters
    nItr = 1e{int(np.log10(nItr))} #iterations
    timeStep = 1 # us, the maximum timestep calculated from kbPN
    timeWrite = 1e5
    pdbWrite = 1e5
    trajWrite = 1e{int(np.log10((nItr/1e1)))}
    assocdissocwrite = true
    overlapSepLimit = 1.0
end parameters

start boundaries
    WaterBox = {WaterBox} # VtoL=4000
end boundaries

start molecules
    P : 3
    N : {len(NCrds)}
    S : {len(SCrds)}
    nuc : {len(nucCrds)}
end molecules

start reactions

    N(ref) + nuc(ref) -> N(ref!1).nuc(ref!1)
    onRate3Dka = 0
    offRatekb = 0
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 5.5
    assocAngles = [nan, nan, nan, nan, nan]
    loopcoopfactor = 1
    bindRadSameCom = 1.1

    P(ref) + nuc(ref) -> P(ref!1).nuc(ref!1)
    onRate3Dka = 0
    offRatekb = 0
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 5.5
    assocAngles = [nan, nan, nan, nan, nan]
    loopcoopfactor = 1
    bindRadSameCom = 1.1    

    S(re1) + S(re2) -> S(re1!1).S(re2!1)
    onRate3Dka = 1e6
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.80001
    assocAngles = [M_PI, M_PI, nan, nan, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.00001

    S(bs) + P(dbs) <-> S(bs!1).P(dbs!1)
    onRate3Dka = {kaPS}
    offRatekb = {kbPS} # Keq = 10^{rlog(1e3*KPN*C0)}
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.00
    assocAngles = [0.785398, 2.356194, 0.000000, M_PI, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10

    N(bs) + P(dbn) <-> N(bs!1).P(dbn!1)
    onRate3Dka = {kaPN}
    offRatekb = {kbPN} # Keq = 10^{rlog(KPN*C0)}
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.000007
    assocAngles = [1.604246, 0.948003, 1.221389, -2.514197, -1.261396]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10
    
    P(pp1) + P(pp2) <-> P(pp1!1).P(pp2!1)
    onRate3Dka = {kaPP} 
    offRatekb = {kbPP} # Keq = 10^{rlog(KPP*C0)}
    norm1 = [0, 0, 1]
    norm2 = [0, 0, 1]
    sigma = 1.4
    assocAngles = [2.111216, 2.111216, -1.570796, 1.570796, 0.000000]
    loopcoopfactor = 1
    bindRadSameCom = 1.1
    area3dto1d = 10

end reactions
'''
        # create directory and write parameters
        Path(namedir).mkdir(parents=True, exist_ok=True)
        with open(f'./{namedir}/parms.inp', 'w') as f:
            f.write(parmFile)
        # write coordinates
        crdFileName = f'./{namedir}/fixCoordinates.inp'
        fixCoords={'S':SCrds,  'N':NCrds, 'nuc':nucCrds}
        Write_CrdInp(crdFileName, fixCoords, WaterBox)

kaP: 0.00e+00 kbP: 0.00e+00 kbN: 1.21e+03 kbS: 6.05e+00
kaP: 6.73e-01 kbP: 4.05e-01 kbN: 1.21e+03 kbS: 6.05e+00
